In [72]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import auc
from sklearn import metrics

## Data Prep

In [73]:
path = '/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/'
veid_data_sample = pd.read_csv(path + 'veid_data.csv')
Browsing_History_Sample = pd.read_csv(path + 'Browsing_History.csv')
DEVICE_DATA_Sample = pd.read_csv(path + 'device_data.csv', encoding='iso-8859-1' )
IP3_APP_sample = pd.read_csv(path + 'IP3_APP.csv')
IP3_CM_sample = pd.read_csv(path + 'IP3_CM.csv')
modelling_sample = pd.read_csv(path + 'Modeling.csv')

print (DEVICE_DATA_Sample.shape, IP3_APP_sample.shape, IP3_CM_sample.shape, modelling_sample.shape)

(22134, 17) (97801, 19) (286654, 139) (106755, 14)


In [74]:
import datetime
modelling_sample['hour'] = modelling_sample['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').hour )
modelling_sample['day'] = modelling_sample['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').day )
modelling_sample['month'] = modelling_sample['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').month )
modelling_sample['weekday'] = modelling_sample['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').weekday() )


In [75]:
modelling_IP3CM_APP = modelling_sample.merge(IP3_CM_sample, on='ip_3', how = 'left').merge(IP3_APP_sample, on='ip_3', how = 'left')
modelling_IP3CM_APP = modelling_IP3CM_APP.fillna(-999)
print (modelling_IP3CM_APP.shape, modelling_sample.shape, IP3_CM_sample.shape, IP3_APP_sample.shape)

(106755, 174) (106755, 18) (286654, 139) (97801, 19)


In [76]:
DEVICE_DATA_Sample = DEVICE_DATA_Sample.rename(columns={'UserAgent':"user_agent"})
modelling_IP3CM_APP_device = modelling_IP3CM_APP.merge(DEVICE_DATA_Sample, on='user_agent', how = 'left')

## Browsing Attributes

In [77]:
modelling_IP3CM_APP_device['new_var1'] = modelling_IP3CM_APP_device['Product_Color_Desc'].map(lambda x: 1 if x == 'GOLD' else 0)
modelling_IP3CM_APP_device['new_var2'] = modelling_IP3CM_APP_device['Product_Color_Desc'].map(lambda x: 1 if x == 'PLATINUM' else 0)
modelling_IP3CM_APP_device['new_var3'] = modelling_IP3CM_APP_device['sub_channel'].map(lambda x: 1 if x == 'Interactive' else 0)

modelling_IP3CM_APP_device['new_var4'] = modelling_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:E77' else 0)
modelling_IP3CM_APP_device['new_var5'] = modelling_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:E82' else 0)
modelling_IP3CM_APP_device['new_var6'] = modelling_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:I73' else 0)

In [112]:
browsing_path = '/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/browsing/'
browsing_data_vars = pd.read_csv(browsing_path + 'browsing_data_vars.csv', header=None)
browsing_data_vars1 = pd.read_csv(browsing_path + 'browsing_data_vars1.csv', header=None)
browsing_data_vars2 = pd.read_csv(browsing_path + 'browsing_data_vars2.csv', header=None)
browsing_data_vars3 = pd.read_csv(browsing_path + 'browsing_data_vars3.csv', header=None)
browsing_data_vars4 = pd.read_csv(browsing_path + 'browsing_data_vars4.csv', header=None)
browsing_data_vars5 = pd.read_csv(browsing_path + 'browsing_data_vars5.csv', header=None)
browsing_data_vars6 = pd.read_csv(browsing_path + 'browsing_data_vars6.csv', header=None)
print (browsing_data_vars.shape, browsing_data_vars1.shape, browsing_data_vars2.shape, browsing_data_vars3.shape, browsing_data_vars4.shape, browsing_data_vars5.shape, browsing_data_vars6.shape )

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-1: malformed \N character escape (<ipython-input-112-03448fc11ab8>, line 2)

In [79]:
browsing_data_vars.columns=[['pcn']+[str(i)+'_br' for i in list(range(0,11))]]
browsing_data_vars1.columns=[['pcn']+[str(i)+'_br1' for i in list(range(0,171))]]
browsing_data_vars2.columns=[['pcn']+[str(i)+'_br2' for i in list(range(0,13))]]
browsing_data_vars3.columns=[['pcn']+[str(i)+'_br3' for i in list(range(0,159))]]
browsing_data_vars4.columns=[['pcn']+[str(i)+'_br4' for i in list(range(0,139))]]
browsing_data_vars5.columns=[['pcn']+[str(i)+'_br5' for i in list(range(0,42))]]
browsing_data_vars6.columns=[['pcn']+[str(i)+'_br6' for i in list(range(0,127))]]

In [80]:
modelling_IP3CM_APP_device_brows = modelling_IP3CM_APP_device.merge(browsing_data_vars, on = 'pcn', how = 'left').merge(browsing_data_vars1, on = 'pcn', how = 'left').merge(browsing_data_vars2,on = 'pcn', how = 'left').merge(browsing_data_vars3, on = 'pcn', how = 'left').merge(browsing_data_vars4, on = 'pcn', how = 'left').merge(browsing_data_vars5,on = 'pcn', how = 'left').merge(browsing_data_vars6, on = 'pcn', how = 'left')

modelling_IP3CM_APP_device_brows.shape

(106755, 858)

## Extra

In [ ]:
# y = pd.DataFrame(modelling_IP3CM_APP_device_brows['8_br'].fillna('-999'))
y=[]
for index, row in modelling_IP3CM_APP_device_brows.iterrows():
    try:
        k = float(row['8_br'])
        y.append(k)
        print (k)
    except ValueError:
        pass

201.88888888888889
52.285714285714285
33.07142857142857
138.86666666666667
209.625
108.0
84.8
154.25
221.33333333333334
109.4
102.125
131.9
139.3125
70.11111111111111
73.375
264.42857142857144
111.85714285714286
64.1
94.57142857142857
90.375
43.25
106.125
63.0
129.89473684210526
68.14285714285714
38.464285714285715
161.625
59.125
57.4
38.0
38.22222222222222
71.85714285714286
65.125
59.0
101.375
129.5
49.8125
86.54545454545455
60.142857142857146
42.666666666666664
115.125
142.0
23.2
56.36363636363637
78.81818181818181
99.77777777777777
63.111111111111114
177.27777777777777
52.125
46.8
48.125
53.875
98.25
82.54545454545455
68.0
44.25
100.7
72.25
127.79166666666667
151.33333333333334
57.55555555555556
74.3076923076923
156.11111111111111
63.5
76.5
160.28571428571428
113.86666666666666
83.0
237.41666666666666
92.26666666666667
57.77777777777778
49.5
47.888888888888886
44.75
84.5
109.4
43.666666666666664
86.16666666666667
211.25
77.25
178.1
44.2
82.0
90.71428571428571
95.25
88.375
58.3333333

In [113]:
print(is_number('foo'))

NameError: name 'is_number' is not defined

In [94]:
#x=y[y['8_br']=='-999']
#y['new_cont1']=y['8_br'].map(lambda x : float(x))
print (y['8_br'].mean())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [41]:
x=modelling_IP3CM_APP_device_brows['Avg_Age']
x = pd.DataFrame(x)
x = x[x['Avg_Age']!=-999]
x['Avg_Age'].mean()

y['']=modelling_IP3CM_APP_device_brows['8_br'].map(lambda x : float(x))

x=modelling_IP3CM_APP_device_brows['Avg_Age']
x = pd.DataFrame(x)
x = x[x['Avg_Age']!=-999]
x['Avg_Age'].mean()

42.317379779267036

In [50]:
modelling_IP3CM_APP_device_brows['8_br'].head()

0    201.88888888888889
1    52.285714285714285
2     33.07142857142857
3    138.86666666666667
4               209.625
Name: 8_br, dtype: object

In [55]:
modelling_IP3CM_APP_device_brows['8_br'].replace('\\N','-999')
y['new_cont1']=modelling_IP3CM_APP_device_brows['8_br'].map(lambda x : try float(x))

ValueError: could not convert string to float: '\\N'

In [44]:
y=modelling_IP3CM_APP_device_brows[modelling_IP3CM_APP_device_brows['Avg_Age']==-999]
y.shape

(2147, 860)

In [34]:
from math import log, floor
modelling_IP3CM_APP_device_brows['new_cont1'] = modelling_IP3CM_APP_device_brows['Avg_Age'].map(lambda x : 42.31 if x==-999 or x==0 else x)
modelling_IP3CM_APP_device_brows['new_cont2'] = modelling_IP3CM_APP_device_brows['new_cont1'].map(lambda x : floor((log(x)*log(x))))

In [47]:
x=modelling_IP3CM_APP_device_brows.groupby('new_cont2')['Gamer_flag'].count()
print (x)

new_cont2
8        16
9        74
10      397
11     3107
12    17660
13    35397
14    28589
15    17077
16     3812
17      521
18       92
19       12
20        1
Name: Gamer_flag, dtype: int64


In [129]:
modelling_IP3CM_APP_device_brows['Avg_Age'].dtypes

dtype('float64')

In [136]:
np.percentile(list(modelling_IP3CM_APP_device_brows['Avg_Age']),100)

89.0

In [104]:
modelling_IP3CM_APP_device_brows.to_csv('/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/sample/modelling_IP3CM_APP_device_brows.csv', index=False)
InterimEvaluation_IP3CM_APP_device_brows.to_csv('/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/sample/finalEvaluation_IP3CM_APP_device_brows.csv', index=False)

In [114]:
InterimEvaluation_IP3CM_APP_device_brows.to_csv('/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/sample/InterimEvaluation_IP3CM_APP_device_brows.csv', index=False)

## Train, Valid and Test data split

In [40]:
## Train, Valid, Test split

import copy
final_data = copy.deepcopy(modelling_IP3CM_APP_device_brows)


pcn  = pd.DataFrame({'pcn':list(set(final_data['pcn']))})
train = pcn.sample(frac=0.7,random_state=1)
valid = set(pcn['pcn'])-set(train['pcn'])
valid = pd.DataFrame({'pcn':list(valid)})
valid = valid.sample(frac=0.67,random_state=1)

test  = set(pcn['pcn'])-set(train['pcn'])-set(valid['pcn'])
test  = pd.DataFrame({'pcn':list(test)})

print(train.shape)
print(valid.shape)
print(test.shape)

print(set(train['pcn'])&set(valid['pcn'])&set(test['pcn']))

train = train.merge(final_data, on=['pcn'], how='left')
valid = valid.merge(final_data, on=['pcn'], how='left')
test  = test.merge(final_data, on=['pcn'], how='left')

print(train.shape)
print(valid.shape)
print(test.shape)

(74728, 1)
(21458, 1)
(10569, 1)
set()
(74728, 858)
(21458, 858)
(10569, 858)


In [41]:
list_drop = ['pcn', 'post_uni_visid', 'datetime', 'Market_cd_x', 'Market_cd_y', 'Gamer_flag']
train_x = train.drop(list_drop, axis=1)
train_y = train['Gamer_flag']

valid_x = valid.drop(list_drop, axis=1)
valid_y = valid['Gamer_flag']

test_x = test.drop(list_drop, axis=1)
test_y = test['Gamer_flag']


list_cat= []
for i in train_x:
    if train_x[i].dtypes == 'object':
        list_cat.append(i)
list_cat

['ip',
 'geo_region',
 'geo_city',
 'product_list',
 'user_agent',
 'Product_Color_Desc',
 'sub_channel',
 'ip_3',
 'DeviceType',
 'Osname',
 'Browsername',
 'primaryHardwareType',
 'model',
 '8_br',
 '9_br']

In [42]:
train_x = train_x.fillna(-999)
valid_x = valid_x.fillna(-999)
test_x = test_x.fillna(-999)
for i in list_cat:
        train_x[i] = train_x[i].map(lambda x : str(x))
        valid_x[i] = valid_x[i].map(lambda x : str(x))
        test_x[i] = test_x[i].map(lambda x : str(x))
        
        le = preprocessing.LabelEncoder()
        train_x[i] = le.fit_transform(train_x[i])
        valid_x[i] = le.fit_transform(valid_x[i])
        test_x[i] = le.fit_transform(test_x[i])

## Final submission data

In [97]:
path = '/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/'
DEVICE_DATA_Sample = pd.read_csv(path + 'device_data.csv', encoding='iso-8859-1' )
IP3_APP_sample = pd.read_csv(path + 'IP3_APP.csv')
IP3_CM_sample = pd.read_csv(path + 'IP3_CM.csv')
modelling_sample = pd.read_csv(path + 'Modeling.csv')
print (DEVICE_DATA_Sample.shape, IP3_APP_sample.shape, IP3_CM_sample.shape, modelling_sample.shape) 

(22134, 17) (97801, 19) (286654, 139) (106755, 14)


In [98]:
path = '/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/'
#InterimEvaluation = pd.read_csv(path + 'interimevaluation_actual.csv')
InterimEvaluation = pd.read_csv(path + 'FinalEvaluation.csv')
print (InterimEvaluation.shape, IP3_APP_sample.shape, IP3_CM_sample.shape, modelling_sample.shape)

(66497, 13) (97801, 19) (286654, 139) (106755, 14)


In [99]:
import datetime
InterimEvaluation['hour'] = InterimEvaluation['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').hour )
InterimEvaluation['day'] = InterimEvaluation['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').day )
InterimEvaluation['month'] = InterimEvaluation['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').month )
InterimEvaluation['weekday'] = InterimEvaluation['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').weekday() )

In [100]:
InterimEvaluation_IP3CM_APP = InterimEvaluation.merge(IP3_CM_sample, on='ip_3', how = 'left').merge(IP3_APP_sample, on='ip_3', how = 'left')
InterimEvaluation_IP3CM_APP = InterimEvaluation_IP3CM_APP.fillna(-999)
print (InterimEvaluation_IP3CM_APP.shape, modelling_sample.shape, IP3_CM_sample.shape, IP3_APP_sample.shape) 

(66497, 173) (106755, 14) (286654, 139) (97801, 19)


In [101]:
DEVICE_DATA_Sample = DEVICE_DATA_Sample.rename(columns={'UserAgent':"user_agent"})
InterimEvaluation_IP3CM_APP_device = InterimEvaluation_IP3CM_APP.merge(DEVICE_DATA_Sample, on='user_agent', how = 'left') 

In [102]:
InterimEvaluation_IP3CM_APP_device['new_var1'] = InterimEvaluation_IP3CM_APP_device['Product_Color_Desc'].map(lambda x: 1 if x == 'GOLD' else 0)
InterimEvaluation_IP3CM_APP_device['new_var2'] = InterimEvaluation_IP3CM_APP_device['Product_Color_Desc'].map(lambda x: 1 if x == 'PLATINUM' else 0)
InterimEvaluation_IP3CM_APP_device['new_var3'] = InterimEvaluation_IP3CM_APP_device['sub_channel'].map(lambda x: 1 if x == 'Interactive' else 0)

InterimEvaluation_IP3CM_APP_device['new_var4'] = InterimEvaluation_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:E77' else 0)
InterimEvaluation_IP3CM_APP_device['new_var5'] = InterimEvaluation_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:E82' else 0)
InterimEvaluation_IP3CM_APP_device['new_var6'] = InterimEvaluation_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:I73' else 0)

In [103]:
InterimEvaluation_IP3CM_APP_device_brows = InterimEvaluation_IP3CM_APP_device.merge(browsing_data_vars, on = 'pcn', how = 'left').merge(browsing_data_vars1, on = 'pcn', how = 'left').merge(browsing_data_vars2,on = 'pcn', how = 'left').merge(browsing_data_vars3, on = 'pcn', how = 'left').merge(browsing_data_vars4, on = 'pcn', how = 'left').merge(browsing_data_vars5,on = 'pcn', how = 'left').merge(browsing_data_vars6, on = 'pcn', how = 'left')

InterimEvaluation_IP3CM_APP_device_brows.shape

(66497, 857)

In [58]:
list_drop = ['pcn', 'post_uni_visid', 'datetime', 'Market_cd_x', 'Market_cd_y']
final_sub = InterimEvaluation_IP3CM_APP_device_brows.drop(list_drop, axis=1)
#final_sub_y = InterimEvaluation_IP3CM_APP_device_brows['Gamer_flag']

list_cat=['ip','geo_region', 'geo_city',
 'product_list',
 'user_agent',
 'Product_Color_Desc',
 'sub_channel',
 'ip_3',
 'DeviceType',
 'Osname',
 'Browsername',
 'primaryHardwareType',
 'model',
 '8_br',
 '9_br']

In [59]:
for i in list_cat:
        print(i)
        final_sub[i] = final_sub[i].map(lambda x : str(x))
        
        le = preprocessing.LabelEncoder()

        final_sub[i] = le.fit_transform(final_sub[i])

ip
geo_region
geo_city
product_list
user_agent
Product_Color_Desc
sub_channel
ip_3
DeviceType
Osname
Browsername
primaryHardwareType
model
8_br
9_br


## LightGBM model

In [84]:
model=lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=35, max_depth=-1, learning_rate=0.005, n_estimators=1500, max_bin=255, subsample_for_bin=50000, objective=None, min_split_gain=0.0, min_child_weight=5, min_child_samples=10, subsample=1.0, subsample_freq=1, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, silent=True)

In [85]:
model.fit(train_x,train_y,
        eval_set=[(valid_x,valid_y)],
        eval_metric='logloss',
        early_stopping_rounds=30) 

[1]	valid_0's binary_logloss: 0.689293
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's binary_logloss: 0.685478
[3]	valid_0's binary_logloss: 0.681701
[4]	valid_0's binary_logloss: 0.677962
[5]	valid_0's binary_logloss: 0.674259
[6]	valid_0's binary_logloss: 0.670594
[7]	valid_0's binary_logloss: 0.666964
[8]	valid_0's binary_logloss: 0.66337
[9]	valid_0's binary_logloss: 0.659813
[10]	valid_0's binary_logloss: 0.656286
[11]	valid_0's binary_logloss: 0.652801
[12]	valid_0's binary_logloss: 0.649342
[13]	valid_0's binary_logloss: 0.645923
[14]	valid_0's binary_logloss: 0.64253
[15]	valid_0's binary_logloss: 0.639174
[16]	valid_0's binary_logloss: 0.635847
[17]	valid_0's binary_logloss: 0.632548
[18]	valid_0's binary_logloss: 0.629288
[19]	valid_0's binary_logloss: 0.626051
[20]	valid_0's binary_logloss: 0.622846
[21]	valid_0's binary_logloss: 0.619672
[22]	valid_0's binary_logloss: 0.616526
[23]	valid_0's binary_logloss: 0.613409
[24]	valid_0's binary_logloss

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1.0,
        learning_rate=0.005, max_bin=255, max_depth=-1,
        min_child_samples=10, min_child_weight=5, min_split_gain=0.0,
        n_estimators=1500, nthread=-1, num_leaves=35, objective='binary',
        reg_alpha=0.0, reg_lambda=0.0, seed=0, silent=True, subsample=1.0,
        subsample_for_bin=50000, subsample_freq=1)

In [45]:
pred_valid =model.predict_proba(valid_x)
pred_test =model.predict_proba(test_x)
pred_train =model.predict_proba(train_x)

In [46]:
dt_valid=pd.DataFrame(pred_valid)
dt_test=pd.DataFrame(pred_test)
dt_train=pd.DataFrame(pred_train)

dt_valid.columns=['0_score','1_score']
dt_test.columns=['0_score','1_score']
dt_train.columns=['0_score','1_score']
print(dt_valid.shape, dt_test.shape, dt_train.shape)

(21458, 2) (10569, 2) (74728, 2)


In [47]:
# Evaluation
from sklearn.metrics import roc_curve, auc
fpr_valid,tpr_valid,thresholds_valid = roc_curve(valid_y,dt_valid['1_score'])
fpr_test,tpr_test,thresholds_test = roc_curve(test_y,dt_test['1_score'])
fpr_train,tpr_train,thresholds_train = roc_curve(train_y,dt_train['1_score'])

print ('valid:', 2*auc(fpr_valid,tpr_valid) - 1, 'test:', 2*auc(fpr_test,tpr_test) - 1,   'train:', 2*auc(fpr_train,tpr_train) - 1)

valid: 0.519938233406 test: 0.47328452518 train: 0.705582925868


In [ ]:
i = list(zip(list(train_x.columns),list(model.feature_importances_)))
sorted(i,key=lambda x :x[1],reverse=True)

In [49]:
train_score_lgbm=train[['pcn','Gamer_flag']].join(dt_train['1_score'])
valid_score_lgbm=valid[['pcn','Gamer_flag']].join(dt_valid['1_score'])
test_score_lgbm=test[['pcn','Gamer_flag']].join(dt_test['1_score'])

train_score_lgbm.to_csv('/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/train_score_lgbm.csv',index=False)
valid_score_lgbm.to_csv('/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/valid_score_lgbm.csv',index=False)
test_score_lgbm.to_csv('/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/test_score_lgbm.csv',index=False)

In [50]:
print (train_score_lgbm['1_score'].mean(), valid_score_lgbm['1_score'].mean(), train_score_lgbm['1_score'].mean())

0.0675989425488 0.0660426531651 0.0675989425488


# XGBoost Model

In [62]:
parameters = {}
parameters['eta']              = 0.01
parameters['max_depth']        = 6
parameters['subsample']        = 0.75
parameters['colsample_bytree'] = 0.75
parameters['objective']        = 'binary:logistic'
parameters['eval_metric']      = 'auc'
parameters['seed']             = 1008
parameters['nthread']          = 20
parameters['tree_method']      ='exact'
parameters['booster']          ='gbtree'

In [63]:
train_matrix = xgb.DMatrix(data=train_x, label=train_y)
valid_matrix = xgb.DMatrix(data=valid_x, label=valid_y)
test_matrix = xgb.DMatrix(data=test_x, label=test_y)
model = xgb.train(params=parameters, dtrain=train_matrix, num_boost_round=300)

In [64]:
pred_valid =model.predict(valid_matrix)
pred_test =model.predict(test_matrix)
pred_train =model.predict(train_matrix)

In [65]:
dt_valid=pd.DataFrame(pred_valid)
dt_test=pd.DataFrame(pred_test)
dt_train=pd.DataFrame(pred_train)

dt_valid.columns=['score']
dt_test.columns=['score']
dt_train.columns=['score']
print(dt_valid.shape, dt_test.shape, dt_train.shape)

(21458, 1) (10569, 1) (74728, 1)


In [66]:
# Evaluation
from sklearn.metrics import roc_curve, auc
fpr_valid,tpr_valid,thresholds_valid = roc_curve(valid_y,dt_valid['score'])
fpr_test,tpr_test,thresholds_test = roc_curve(test_y,dt_test['score'])
fpr_train,tpr_train,thresholds_train = roc_curve(train_y,dt_train['score'])

print ('valid:', 2*auc(fpr_valid,tpr_valid) - 1, 'test:', 2*auc(fpr_test,tpr_test) - 1,   'train:', 2*auc(fpr_train,tpr_train) - 1)

valid: 0.508763176832 test: 0.470329066738 train: 0.619023400554


In [ ]:
i=[(k,v) for k,v in model.get_fscore().items()]
sorted(i,key=lambda x :x[1],reverse=True)

In [67]:
train_score_xgb=train[['pcn', 'Gamer_flag']].join(dt_train['score'])
valid_score_xgb=valid[['pcn', 'Gamer_flag']].join(dt_valid['score'])
test_score_xgb=test[['pcn', 'Gamer_flag']].join(dt_test['score'])

train_score_xgb.to_csv('/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/train_score_xgb.csv',index=False)
valid_score_xgb.to_csv('/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/valid_score_xgb.csv',index=False)
test_score_xgb.to_csv('/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/test_score_xgb.csv',index=False)

In [68]:
print (train_score_xgb['score'].mean(), valid_score_xgb['score'].mean(), test_score_xgb['score'].mean())

0.0881099 0.0875764 0.0886015


## Prediction on Final data 

#### using LightGBM 

In [60]:
dt_final_sub =model.predict_proba(final_sub)
dt_final_sub=pd.DataFrame(dt_final_sub)
dt_final_sub.columns=['0_score','1_score']
print(dt_final_sub.shape)

(66497, 2)


In [ ]:
x=InterimEvaluation[['pcn']].join(dt_final_sub['1_score'])
x.to_csv('/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/GGN23_Final_score.csv',index=False)

#### using xgboost

In [124]:
final_sub_matrix = xgb.DMatrix(data=final_sub)
dt_final_sub =model.predict(final_sub_matrix)
dt_final_sub=pd.DataFrame(dt_final_sub)
dt_final_sub.columns=['score']

In [125]:
x=InterimEvaluation[['pcn']].join(dt_final_sub['score'])
x.to_csv('/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/GGN23_Final_score_xgb.csv',index=False)
x['score'].mean()

## Interim Solution

In [105]:
path = '/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/'
DEVICE_DATA_Sample = pd.read_csv(path + 'device_data.csv', encoding='iso-8859-1' )
IP3_APP_sample = pd.read_csv(path + 'IP3_APP.csv')
IP3_CM_sample = pd.read_csv(path + 'IP3_CM.csv')
modelling_sample = pd.read_csv(path + 'Modeling.csv')
print (DEVICE_DATA_Sample.shape, IP3_APP_sample.shape, IP3_CM_sample.shape, modelling_sample.shape) 

(22134, 17) (97801, 19) (286654, 139) (106755, 14)


In [106]:
path = '/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/'
InterimEvaluation = pd.read_csv(path + 'interimevaluation_actual.csv')
print (InterimEvaluation.shape, IP3_APP_sample.shape, IP3_CM_sample.shape, modelling_sample.shape)

(45751, 14) (97801, 19) (286654, 139) (106755, 14)


In [107]:
import datetime
InterimEvaluation['hour'] = InterimEvaluation['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').hour )
InterimEvaluation['day'] = InterimEvaluation['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').day )
InterimEvaluation['month'] = InterimEvaluation['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').month )
InterimEvaluation['weekday'] = InterimEvaluation['datetime'].map(lambda x : datetime.datetime.strptime(x, '%d%b%Y:%H:%M:%S').weekday() )

In [108]:
InterimEvaluation_IP3CM_APP = InterimEvaluation.merge(IP3_CM_sample, on='ip_3', how = 'left').merge(IP3_APP_sample, on='ip_3', how = 'left')
InterimEvaluation_IP3CM_APP = InterimEvaluation_IP3CM_APP.fillna(-999)
print (InterimEvaluation_IP3CM_APP.shape, modelling_sample.shape, IP3_CM_sample.shape, IP3_APP_sample.shape) 

(45751, 174) (106755, 14) (286654, 139) (97801, 19)


In [109]:
DEVICE_DATA_Sample = DEVICE_DATA_Sample.rename(columns={'UserAgent':"user_agent"})
InterimEvaluation_IP3CM_APP_device = InterimEvaluation_IP3CM_APP.merge(DEVICE_DATA_Sample, on='user_agent', how = 'left') 

In [110]:
InterimEvaluation_IP3CM_APP_device['new_var1'] = InterimEvaluation_IP3CM_APP_device['Product_Color_Desc'].map(lambda x: 1 if x == 'GOLD' else 0)
InterimEvaluation_IP3CM_APP_device['new_var2'] = InterimEvaluation_IP3CM_APP_device['Product_Color_Desc'].map(lambda x: 1 if x == 'PLATINUM' else 0)
InterimEvaluation_IP3CM_APP_device['new_var3'] = InterimEvaluation_IP3CM_APP_device['sub_channel'].map(lambda x: 1 if x == 'Interactive' else 0)
InterimEvaluation_IP3CM_APP_device['new_var4'] = InterimEvaluation_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:E77' else 0)
InterimEvaluation_IP3CM_APP_device['new_var5'] = InterimEvaluation_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:E82' else 0)
InterimEvaluation_IP3CM_APP_device['new_var6'] = InterimEvaluation_IP3CM_APP_device['product_list'].map(lambda x: 1 if x == ';UK:ICSS:I73' else 0)

In [113]:
InterimEvaluation_IP3CM_APP_device_brows = InterimEvaluation_IP3CM_APP_device.merge(browsing_data_vars, on = 'pcn', how = 'left').merge(browsing_data_vars1, on = 'pcn', how = 'left').merge(browsing_data_vars2,on = 'pcn', how = 'left').merge(browsing_data_vars3, on = 'pcn', how = 'left').merge(browsing_data_vars4, on = 'pcn', how = 'left').merge(browsing_data_vars5,on = 'pcn', how = 'left').merge(browsing_data_vars6, on = 'pcn', how = 'left')
InterimEvaluation_IP3CM_APP_device_brows.shape

(45751, 858)

In [112]:
list_drop = ['pcn', 'post_uni_visid', 'datetime', 'Market_cd_x', 'Market_cd_y', 'Gamer_flag']
final_sub = InterimEvaluation_IP3CM_APP_device_brows.drop(list_drop, axis=1)
final_sub_y = InterimEvaluation_IP3CM_APP_device_brows['Gamer_flag']

In [77]:
list_cat=['ip','geo_region', 'geo_city',
 'product_list',
 'user_agent',
 'Product_Color_Desc',
 'sub_channel',
 'ip_3',
 'DeviceType',
 'Osname',
 'Browsername',
 'primaryHardwareType',
 'model',
 '8_br',
 '9_br']

In [78]:
for i in list_cat:
        print(i)
        final_sub[i] = final_sub[i].map(lambda x : str(x))
        
        le = preprocessing.LabelEncoder()

        final_sub[i] = le.fit_transform(final_sub[i])

ip
geo_region
geo_city
product_list
user_agent
Product_Color_Desc
sub_channel
ip_3
DeviceType
Osname
Browsername
primaryHardwareType
model
8_br
9_br


### Interim Prediction

#### using lgbm

In [86]:
dt_final_sub =model.predict_proba(final_sub)
dt_final_sub=pd.DataFrame(dt_final_sub)
dt_final_sub.columns=['0_score','1_score']
print(dt_final_sub.shape)

(45751, 2)


In [87]:
x=InterimEvaluation[['pcn', 'Gamer_flag']].join(dt_final_sub['1_score'])
x.to_csv('/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/GGN23_interim_lgbm.csv',index=False)
x['1_score'].mean()

0.067806631950013699

#### using xgboost

In [80]:
final_sub_matrix = xgb.DMatrix(data=final_sub)
dt_final_sub =model.predict(final_sub_matrix)
dt_final_sub=pd.DataFrame(dt_final_sub)
dt_final_sub.columns=['score']

In [81]:
x=InterimEvaluation[['pcn', 'Gamer_flag']].join(dt_final_sub['score'])
x.to_csv('/axp/rim/imsads/dev/sagra39/IMS_Hackathon_17/input/GGN23_interim_xgb.csv',index=False)
x['score'].mean()

0.08796303